In [1]:
# plot the hydrograph from all basins

In [21]:
import pandas as pd
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import xarray as xr
import cftime
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings('ignore')


# observed streamflow
def read_CAMELS_Q(file_Qobs):
    df_q_in = pd.read_csv(file_Qobs, delim_whitespace=True, header=None)
    years = df_q_in[1].values
    months = df_q_in[2].values
    days = df_q_in[3].values
    dates = [f'{years[i]}-{months[i]:02}-{days[i]:02}' for i in range(len(years))]
    dates = pd.to_datetime(dates)
    q_obs = df_q_in[4].values * 0.028316847  # cfs to cms
    q_obs[q_obs < 0] = -9999.0
    df_q_out = pd.DataFrame({'Date': dates, 'Runoff_cms': q_obs})
    
    # fill possible missing values
    df_q_out.set_index('Date', inplace=True)
    date_range = pd.date_range(start=dates[0], end=dates[-1], freq='D')
    df_q_out = df_q_out.reindex(date_range)
    df_q_out.fillna(-9999, inplace=True)
    df_q_out.reset_index(inplace=True)
    df_q_out = df_q_out.rename(columns={'index': 'Date'})

    return df_q_out
    


# Convert the datetime64 time axis of ds_qobs to cftime.DatetimeNoLeap
def convert_time_to_cftime(ds):
    # First, convert the datetime64 objects to pandas Timestamps to access year, month, etc.
    time_values = pd.to_datetime(ds['time'].values)

    # Convert to cftime.DatetimeNoLeap objects
    cftime_no_leap_time = [cftime.DatetimeNoLeap(t.year, t.month, t.day, t.hour, t.minute, t.second)
                           for t in time_values]
    
    # Replace the time coordinate in ds with the new cftime time axis
    ds = ds.assign_coords(time=cftime_no_leap_time)
    
    return ds

def read_flow_data(basin, df_info):
    # basin = 1
    flag = 'iter0_trial0'
    # year=2010
    year='' # all years
    
    file = f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/disaggregation/surfdata_CAMELS_level1_hist_78pfts_CMIP6_simyr2000_HAND_trapezoidal_{basin}.nc'
    ds_surf = xr.load_dataset(file)
    mesharea = ds_surf.AREA.values
    
    # mosart river flow
    inpath = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_{basin}_calib/ctsm_outputs/{flag}/rof/hist'
    files = glob.glob(f'{inpath}/level1_*.mosart.h1.{year}*.nc')
    files.sort()
    ds_mosart = xr.open_mfdataset(files, decode_times=True)
    mean_values = ds_mosart.RIVER_DISCHARGE_OVER_LAND_LIQ.mean(dim='time')
    m1, m2 = np.unravel_index(np.nanargmax(mean_values.values), mean_values.shape)  # Get the indices
    ds_mosart = ds_mosart['RIVER_DISCHARGE_OVER_LAND_LIQ'].isel(lat=m1, lon=m2)
    ds_mosart = ds_mosart.shift(time=-1)
    
    # clm runoff
    inpath = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_{basin}_calib/ctsm_outputs/{flag}/lnd/hist'
    files = glob.glob(f'{inpath}/level1_*.clm2.h1.{year}*.nc')
    files.sort()
    ds_clm = xr.open_mfdataset(files, decode_times=True)
    ds_clm = ds_clm[['QRUNOFF']]
    ds_clm['QRUNOFF'].values = (ds_clm['QRUNOFF'].values / 1000) * (mesharea * 1e6)  # Convert to correct units
    ds_clm = ds_clm.QRUNOFF
    
    # mizuroute runoff
    inpath = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_{basin}_calib/ctsm_outputs/{flag}/mizuroute/'
    files = glob.glob(f'{inpath}/sflow.h*.nc')
    files.sort()
    ds_mizu = xr.open_mfdataset(files, decode_times=True)
    ind = np.argmax(ds_mizu['DWroutedRunoff'].mean(dim='time').values)
    ds_mizu = ds_mizu.isel(seg=ind)
    ds_mizu = ds_mizu.DWroutedRunoff
    ds_mizu = ds_mizu.shift(time=-1)
    
    # obs
    id = df_info.iloc[basin]['hru_id']
    file = f'/glade/work/guoqiang/CTSM_CAMELS/CAMLES_Qobs/{id:08}_streamflow_qc.txt'
    df_qobs = read_CAMELS_Q(file)

    df_qobs = df_qobs[ (df_qobs['Date']>= ds_mizu.time.values[0].strftime('%Y-%m-%d')) & (df_qobs['Date']<= ds_mizu.time.values[-1].strftime('%Y-%m-%d')) ]

    df_qobs['Date'] = pd.to_datetime(df_qobs['Date'])  # Ensure Date is in datetime format
    df_qobs = df_qobs[~((df_qobs['Date'].dt.month == 2) & (df_qobs['Date'].dt.day == 29))]
    df_qobs

    # Convert df_qobs to xarray DataArray
    d = df_qobs['Runoff_cms'].values
    d[d<0]=np.nan
    da_qobs = xr.DataArray(d,
                           dims='time',
                           coords={'time': df_qobs['Date'].values},
                           name='Runoff_cms')
    da_qobs = convert_time_to_cftime(da_qobs)
    
    return ds_clm, ds_mosart, ds_mizu, da_qobs

def calculate_monthly_means(ds):
    # (1) Monthly mean for each month (e.g., Jan 2000, Feb 2000, ..., Dec 2010)
    monthly_mean = ds.resample(time='1M').mean()

    # (2) Monthly climatology (average over all Januarys, all Februarys, etc.)
    monthly_climatology = ds.groupby('time.month').mean('time')
    
    return monthly_mean, monthly_climatology



In [22]:
# basin info
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
df_info = pd.read_csv(file)

file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info_supplment.csv'
df_info2 = pd.read_csv(file)

df_info['name'] = df_info2['station_nm']

In [19]:
# # codes plotting one basin

# basin = 0
# ds_clm, ds_mosart, ds_mizu, ds_qobs = read_flow_data(basin, df_info)
# ds_clm_m1, ds_clm_m2 = calculate_monthly_means(ds_clm)
# ds_mosart_m1, ds_mosart_m2 = calculate_monthly_means(ds_mosart)
# ds_mizu_m1, ds_mizu_m2 = calculate_monthly_means(ds_mizu)
# ds_qobs_m1, ds_qobs_m2 = calculate_monthly_means(ds_qobs)


# files=[
#     "evaluation_many_metrics.csv",
#     "evaluation_many_metrics_mosart_s-1.csv", # shift time step by -1
#     "evaluation_many_metrics_mizuroute_s-1.csv", # shift time step by -1
#     ]
# path0 = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator'
# types = ['CLM', 'MOSART_MaxGrid_s-1', 'mizuroute_s-1', ]
# kge = []
# for j in range(len(types)):
#     fileij = f'{path0}/level1_{basin}_calib/ctsm_outputs/iter0_trial0/{files[j]}'
#     dfij = pd.read_csv(fileij)
#     kge.append(dfij['kge'].values[0])


# fig = plt.figure(figsize=[10, 10])

# fig.add_subplot(3,2,1)
# plt.scatter(df_info['lon_cen'], df_info['lat_cen'], 20, df_info['elev_mean'])
# plt.colorbar()
# plt.scatter(df_info['lon_cen'][basin], df_info['lat_cen'][basin], 30, color='r')
# plt.title('Location')

# fig.add_subplot(3,2,2)
# plt.axis('off')
# loc = np.arange(1, 0, -0.1)
# flag = 0
# for v in ['hru_id', 'name', 'lat_cen', 'lon_cen', 'AREA']:
#     s = df_info.iloc[basin][v]
#     plt.text(0.2, loc[flag], f'{v}: {s}')
#     flag = flag + 1

# for i in range(3):
#     plt.text(0.2, loc[flag], f"{types[i]} KGE': {kge[i]:.2f}")
#     flag = flag+1


# fig.add_subplot(3,2,3)
# ds_qobs.plot(color='k', label='Obs')
# ds_clm.plot(color='g', label='CLM')
# ds_mizu.plot(color='b', label='mizuRoute')
# ds_mosart.plot(color='r', label='MOSART')
# plt.legend(loc='upper left')
# plt.xlabel('')
# plt.ylabel('Streamflow [m3/s]')
# plt.title('Daily series')


# fig.add_subplot(3,2,4)
# ds_qobs.isel(time=slice(0, 365)).plot(color='k', label='Obs')
# ds_clm.isel(time=slice(0, 365)).plot(color='g', label='CLM')
# ds_mizu.isel(time=slice(0, 365)).plot(color='b', label='mizuRoute')
# ds_mosart.isel(time=slice(0, 365)).plot(color='r', label='MOSART')
# plt.legend(loc='upper left')
# plt.xlabel('')
# plt.ylabel('Streamflow [m3/s]')
# plt.title('Daily series in 1st year')


# fig.add_subplot(3,2,5)
# ds_qobs_m1.plot(color='k', label='Obs')
# ds_clm_m1.plot(color='g', label='CLM')
# ds_mizu_m1.plot(color='b', label='mizuRoute')
# ds_mosart_m1.plot(color='r', label='MOSART')
# plt.legend(loc='upper left')
# plt.xlabel('')
# plt.ylabel('Streamflow [m3/s]')
# plt.title('Monthly series')

# fig.add_subplot(3,2,6)
# ds_qobs_m2.plot(color='k', label='Obs')
# ds_clm_m2.plot(color='g', label='CLM')
# ds_mizu_m2.plot(color='b', label='mizuRoute')
# ds_mosart_m2.plot(color='r', label='MOSART')
# plt.legend(loc='upper left')
# plt.xlabel('Month')
# plt.ylabel('Streamflow [m3/s]')
# plt.title('Seasonal flow')


# plt.tight_layout()
# plt.show()

In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Function to generate KGE metrics for the basin
def get_kge_metrics(basin, path0, files, types):
    kge = []
    for j in range(len(types)):
        fileij = f'{path0}/level1_{basin}_calib/ctsm_outputs/iter0_trial0/{files[j]}'
        if os.path.isfile(fileij):
            dfij = pd.read_csv(fileij)
            kge.append(dfij['kge'].values[0])
        else:
            kge.append(np.nan)
    return kge

# Function to read flow data and calculate monthly means
def process_basin_data(basin, df_info):
    # Read flow data
    ds_clm, ds_mosart, ds_mizu, ds_qobs = read_flow_data(basin, df_info)
    
    # Calculate monthly means
    ds_clm_m1, ds_clm_m2 = calculate_monthly_means(ds_clm)
    ds_mosart_m1, ds_mosart_m2 = calculate_monthly_means(ds_mosart)
    ds_mizu_m1, ds_mizu_m2 = calculate_monthly_means(ds_mizu)
    ds_qobs_m1, ds_qobs_m2 = calculate_monthly_means(ds_qobs)
    
    return ds_clm, ds_mosart, ds_mizu, ds_qobs, ds_clm_m1, ds_clm_m2, ds_mosart_m1, ds_mosart_m2, ds_mizu_m1, ds_mizu_m2, ds_qobs_m1, ds_qobs_m2



# Function to plot the results for each basin
def plot_basin_data(basin, df_info, ds_clm, ds_mosart, ds_mizu, ds_qobs, 
                    ds_clm_m1, ds_clm_m2, ds_mosart_m1, ds_mosart_m2, 
                    ds_mizu_m1, ds_mizu_m2, ds_qobs_m1, ds_qobs_m2, kge, types):
    
    fig = plt.figure(figsize=[10, 10])

    # Plot location
    fig.add_subplot(3, 2, 1)
    plt.scatter(df_info['lon_cen'], df_info['lat_cen'], 20, df_info['elev_mean'])
    plt.colorbar()
    plt.scatter(df_info['lon_cen'][basin], df_info['lat_cen'][basin], 30, color='r')
    plt.title('Location')

    # Basin information and KGE values
    fig.add_subplot(3, 2, 2)
    plt.axis('off')
    loc = np.arange(1, 0, -0.1)
    flag = 0
    for v in ['hru_id', 'name', 'lat_cen', 'lon_cen', 'AREA']:
        s = df_info.iloc[basin][v]
        plt.text(0.2, loc[flag], f'{v}: {s}')
        flag = flag + 1

    for i in range(3):
        plt.text(0.2, loc[flag], f"{types[i]} KGE': {kge[i]:.2f}")
        flag = flag + 1

    # Daily series plot
    fig.add_subplot(3, 2, 3)
    ds_qobs.plot(color='k', label='Obs')
    ds_clm.plot(color='g', label='CLM')
    ds_mizu.plot(color='b', label='mizuRoute')
    ds_mosart.plot(color='r', label='MOSART')
    plt.legend(loc='upper left')
    plt.xlabel('')
    plt.ylabel('Streamflow [m3/s]')
    plt.title('Daily series')

    # Daily series in 1st year
    fig.add_subplot(3, 2, 4)
    ds_qobs.isel(time=slice(0, 365)).plot(color='k', label='Obs')
    ds_clm.isel(time=slice(0, 365)).plot(color='g', label='CLM')
    ds_mizu.isel(time=slice(0, 365)).plot(color='b', label='mizuRoute')
    ds_mosart.isel(time=slice(0, 365)).plot(color='r', label='MOSART')
    plt.legend(loc='upper left')
    plt.xlabel('')
    plt.ylabel('Streamflow [m3/s]')
    plt.title('Daily series in 1st year')

    # Monthly series plot
    fig.add_subplot(3, 2, 5)
    ds_qobs_m1.plot(color='k', label='Obs')
    ds_clm_m1.plot(color='g', label='CLM')
    ds_mizu_m1.plot(color='b', label='mizuRoute')
    ds_mosart_m1.plot(color='r', label='MOSART')
    plt.legend(loc='upper left')
    plt.xlabel('')
    plt.ylabel('Streamflow [m3/s]')
    plt.title('Monthly series')

    # Seasonal flow plot (climatology)
    fig.add_subplot(3, 2, 6)
    ds_qobs_m2.plot(color='k', label='Obs')
    ds_clm_m2.plot(color='g', label='CLM')
    ds_mizu_m2.plot(color='b', label='mizuRoute')
    ds_mosart_m2.plot(color='r', label='MOSART')
    plt.legend(loc='upper left')
    plt.xlabel('Month')
    plt.ylabel('Streamflow [m3/s]')
    plt.title('Seasonal flow')

    # Adjust layout
    plt.tight_layout()
    
    return fig

# Main function to loop over basins and save the results to a single PDF
def save_all_plots_to_pdf(output_pdf, df_info, path0, files, types):
    with PdfPages(output_pdf) as pdf:
        for basin in range(0, 627):  # Loop from basin 0 to 627

            print('processing', basin)
            
            # Get the KGE metrics for this basin
            kge = get_kge_metrics(basin, path0, files, types)

            # if kge[2] - kge[1] < -0.5:
            if True:

                try:
                    # Read and process the basin data
                    ds_clm, ds_mosart, ds_mizu, ds_qobs, ds_clm_m1, ds_clm_m2, ds_mosart_m1, ds_mosart_m2, ds_mizu_m1, ds_mizu_m2, ds_qobs_m1, ds_qobs_m2 = process_basin_data(basin, df_info)
                    
                    # Plot the data
                    fig = plot_basin_data(basin, df_info, ds_clm, ds_mosart, ds_mizu, ds_qobs, 
                                          ds_clm_m1, ds_clm_m2, ds_mosart_m1, ds_mosart_m2, 
                                          ds_mizu_m1, ds_mizu_m2, ds_qobs_m1, ds_qobs_m2, kge, types)
                    
                    # Save the figure to the PDF
                    pdf.savefig(fig)
                    plt.close(fig)  # Close the figure after saving
                except:
                    print('Failed basin', basin)
                    
# Set up file paths and types
files = [
    "evaluation_many_metrics.csv",
    "evaluation_many_metrics_mosart_s-1.csv",  # shift time step by -1
    "evaluation_many_metrics_mizuroute_s-1.csv"  # shift time step by -1
]
types = ['CLM', 'MOSART_MaxGrid_s-1', 'mizuroute_s-1']
path0 = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator'

# Output PDF file
# output_pdf = 'camels627_clm_routing_hydrographs_mizuLTmosart.pdf'
output_pdf = 'camels627_clm_routing_hydrographs.pdf'

# Save all plots to one PDF
save_all_plots_to_pdf(output_pdf, df_info, path0, files, types)

print(f"All plots saved to {output_pdf}")


processing 0
processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
processing 17
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
processing 28
processing 29
processing 30
processing 31
processing 32
processing 33
processing 34
processing 35
processing 36
processing 37
processing 38
processing 39
processing 40
processing 41
processing 42
processing 43
processing 44
processing 45
processing 46
processing 47
processing 48
processing 49
processing 50
processing 51
processing 52
processing 53
processing 54
processing 55
processing 56
processing 57
processing 58
processing 59
processing 60
processing 61
processing 62
processing 63
processing 64
processing 65
processing 66
processing 67
processing 68
processing 69
processing 70
processing 71
pr